# Riot Games API
----

API: https://developer.riotgames.com/

Documentation: https://readthedocs.org/projects/riot-api-libraries/downloads/pdf/latest/

### Rate Limits
 - 20 requests every 1 seconds(s)
 - 100 requests every 2 minutes(s)

Due to this all of the pull requests will need to be limited to one pull every 2 seconds which will give enough of a buffer to prevent the servers from denying access.

In [101]:
import cassiopeia as cass
import requests
import json
import ujson

import datetime
import time

import pandas as pd
import numpy as np
from collections import Counter

## Functions
----
Functions

In [126]:
######################################################################
#                             getAPI_key                             #
######################################################################
def getAPI_key():
    """returns Ambisinisterr's Riot API Key from api_key.txt"""
    with open("api_key.txt", "r") as f:
        return f.read()
######################################################################
#                         setAPI_key (cass)                          #
######################################################################
def setAPI_key():
    """Sets the API Key for the Cassiopea API"""
    settings = cass.get_default_config()
    settings["pipeline"]["RiotAPI"]["api_key"] = getAPI_key()
    cass.apply_settings(settings)
######################################################################
#                       get_challenger_players                       #
######################################################################   
def get_challenger_players():
    """Returns a list of all current Challenger Players from the
    Cassiopea API. There is no straight-forward means to do this
    within the official API."""
    data = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives)
    print(challenger_league[0].summoner)
######################################################################
#                     get_account_id_info                            #
######################################################################
def get_account_id_info(summoner_name, region="na1", delay=True):
    """Returns a JSON object from Riot's Summoner-V4 API.
    Requires a summoner_name. Returns players account info such as puuid's.
    Defaults region is NA1. See documentation for other options.
    Defaults to delay 2 seconds before returning in order to conform to rate limits.
    """
    
    key = f"api_key={API_KEY}"
    path = "lol/summoner/v4/summoners/by-name"
    url= f"https://{region}.api.riotgames.com/{path}/{summoner_name}?{key}"
    
    if delay:
        time.sleep(2)
        
    response = requests.get(url)
    return response.json()
######################################################################
#                    get_match_history_json                          #
######################################################################
def get_match_history_json(puuid, count=100, queue_type="ranked",
                           region="americas", delay=True):
    """Returns a JSON object from Riot's Match-V5 API.
    Requires a puuid. Returns {count} matchId's.
    Default count is 100. The count must be 0-100.
    Default queue_type is ranked. See documentation for other options.
    Default region is americas. See documentation for other options.
    Defaults to delay 2 seconds before returning in order to conform to rate limits.
    """
    key = f"api_key={API_KEY}"
    path = "lol/match/v5/matches/by-puuid"
    ids = f"ids?type={queue_type}&start=0&count={count}"
    url = f"https://{region}.api.riotgames.com/{path}/{puuid}/{ids}&{key}"
    
    if delay:
        time.sleep(2)
        
    response = requests.get(url)
    return response.json()
######################################################################
#                                                                    #
######################################################################
def get_match_json(matchid, region="americas", delay=True):
    """Returns a JSON object from Riot's Match-V5 API.
    Requires a matchId. Returns detailed info on the match.
    Default region is americas. See documentation for other options.
    Defaults to delay 2 seconds before returning in order to conform to rate limits.
    """
    key = f"api_key={API_KEY}"
    path = "/lol/match/v5/matches/"
    url = f"https://{region}.api.riotgames.com{path}{matchid}?{key}"
    
    if delay:
        time.sleep(2)
        
    response = requests.get(url)
    return response.json()
######################################################################
#                      estimated_completion_time                     #
######################################################################   
def estimated_completion_time(timedelta_in_seconds):
    """Returns a Datetime Object of a Timestamp in the future based on
    a time delta."""
    now = datetime.datetime.now()
    completion_time = datetime.timedelta(seconds=timedelta_in_seconds)
    return (now + completion_time).strftime("%H:%M:%S")

def get_player_id(number):
    """Returns the string value of the player/participant's index value
    string format is t#p#
    team 1 is index [0:4], team 2 is [5:9] noted as t1 or t2
    player numbers are 1-5 for [0:4] and 1-5 for [5:9], respectively.
    """
    player_id = {0 : "t1p1",
                 1 : "t1p2",
                 2 : "t1p3",
                 3 : "t1p4",
                 4 : "t1p5",
                 5 : "t2p1",
                 6 : "t2p2",
                 7 : "t2p3",
                 8 : "t2p4",
                 9 : "t2p5"}
    return player_id[number]

In [127]:
######################################################################
#                    append_single_match_features                    #
######################################################################
def append_single_match_features(json_index, match_template):
    """Appends the values of the match_json at a certain index."""
    match_data = match_json[json_index]["metadata"]["matchId"]
    players = match_json[json_index]["info"]["participants"]
    teams = match_json[json_index]["info"]["teams"]
    
    match_template["match_id"].append(match_json[json_index]["metadata"]["matchId"])
    match_template["game_duration"].append(match_json[json_index]["info"]["gameDuration"])
    match_template["game_mode"].append(match_json[json_index]["info"]["gameMode"])
    match_template["game_type"].append(match_json[json_index]["info"]["gameType"])
    match_template["game_version"].append(match_json[json_index]["info"]["gameVersion"])
    
    for i, player in enumerate(players):
        pid = get_player_id(i)
        for feature in PLAYER_FEATURES:
            match_template[f"{pid}_{feature}"].append(player[feature])
            
        statPerks = player["perks"]["statPerks"]
        for statPerk in statPerks:
            match_template[f"{pid}_statPerk_{statPerk}"].append(statPerks[statPerk])
            
        styles = player["perks"]["styles"]
        match_template[f"{pid}_perk_primaryStyle"].append(styles[0]["style"])
        match_template[f"{pid}_perk_keystone"].append(styles[0]["selections"][0]["perk"])
        match_template[f"{pid}_perk_primary_slot_1"].append(styles[0]["selections"][1]["perk"])
        match_template[f"{pid}_perk_primary_slot_2"].append(styles[0]["selections"][2]["perk"])
        match_template[f"{pid}_perk_primary_slot_3"].append(styles[0]["selections"][3]["perk"])
        match_template[f"{pid}_perk_subStyle"].append(styles[1]["style"])
        match_template[f"{pid}_perk_sub_slot_1"].append(styles[1]["selections"][0]["perk"])
        match_template[f"{pid}_perk_sub_slot_2"].append(styles[1]["selections"][1]["perk"])
    
    for i, team in enumerate(teams):
        for j, ban in enumerate(team["bans"]):
            match_template[f"t{i+1}_ban{j+1}"].append(teams[i]["bans"][j])
        for objective in team["objectives"]:
            team_objective = teams[i]["objectives"][objective]
            match_template[f"t{i+1}_{objective}_first"].append(int(team_objective["first"]))
            match_template[f"t{i+1}_{objective}_kills"].append(int(team_objective["kills"]))
            
    match_template["t1_win"].append(int(teams[0]["win"]))
    
            
    return match_template
######################################################################
#                       get_all_match_features                       #
######################################################################
def get_all_match_features(json_object, qty=-1):
    """Returns a set of key:value pairs that can be put into a Dataframe Format.
    Itterates through the list of the matchs in the JSON Object of matches.
    
    Defaults to using all match elements in the JSON object.
    If qty is greater than the number of match elements function returns all
    match elements.
    
    JSON Object must be a list of key:value pairs. If only one is being pulled
    use append_single_match_features instead."""
    
    # if JSON Object is not a list of objects, raise exception
    if not isinstance(json_object, list):
        print("Error: JSON Object is not a list.")
        raise NameError("Error: JSON Object is not a list.")
    
    # pull the blank template of all columns of the final df.
    with open("./data/matches/match_template.json", "r") as f:
        match_template = json.load(f)
    
    # run through all elements if qty is -1 or larger than the json length
    if (qty == -1) or (qty > len(json_object)):
        qty = len(json_object)
    
    # There were some failures at times. Prints index value for debugging purposes.
    for i in range(qty):
        try:
            match_template = append_single_match_features(i, match_template)
        except:
            print(f"Failed on {i}")

    return match_template

In [104]:
def verify_API_key():
    path = "lol/summoner/v4/summoners/by-name/Ambisinisterr"
    url = f"https://na1.api.riotgames.com/{path}?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 403:
        print("Status Code 403. Verify API Key.")
    if not response.ok:
        raise NameError(f"Error: Status Code: {response.status_code}")
    else:
        print(f"Response ok! Code: {response.status_code}")

Global Constant `PLAYER_FEATURES` is a list of all the participant features that will be desired for the final dataset.

In [152]:
global MATCH_FEATURES, PLAYER_FEATURES, TEAM_FEATURES, PLAYER_PREFIXES, API_KEY
MATCH_FEATURES = list(pd.read_csv("./data/constants/match_features.csv")["features"])
PLAYER_FEATURES = list(pd.read_csv("./data/constants/player_features.csv")["features"])
TEAM_FEATURES = list(pd.read_csv("./data/constants/team_features.csv")["features"])
PLAYER_PREFIXES = list(pd.read_csv("./data/constants/player_prefixes.csv")["prefix"])
API_KEY = getAPI_key()

## API Key Test
----

In [106]:
verify_API_key()

Response ok! Code: 200


# Cassiopeia
----
Cassiopeia is a Third Party League of Legends API. This was used only to get updated Challenger Ranked Players to pull match history for. Instead of installing Cassiopeia this section of code can be skipped.

In [107]:
cass.set_riot_api_key(API_KEY)

In [108]:
challenger_league = cass.get_challenger_league(region="NA", queue=cass.Queue.ranked_solo_fives)

In [109]:
challenger_names = [player.summoner.name for player in challenger_league]

# Official Riot API
----
Only the Riot Summoner-V4 API allows searching with publicly facing information such as Summoner (account display) Name. The other API's require Riot's internal identification information such as `id`, `puuid` or `accountId`.

The first step to collect Match Information is to take the list of the Challenger Ranked Players and pull their internal identification information from the Summoner-V4 API.

In [110]:
accounts = pd.read_csv("./data/accounts/accounts_with_ids.csv")

In [111]:
# return the challengers account information if not already in the accounts df
new_summoner_info = [get_account_id_info(player) for player in challenger_names
                                      if player not in accounts["name"].values]
# ensure all of the queries were successful
new_summoner_info = [info for info in new_summoner_info if isinstance(info, str)]
print(f"There are {len(new_summoner_info)} new challengers") 

There are 0 new challengers


## Update Account DataFrame
----

In [112]:
# creates a dataframe of the new account information
if len(new_summoner_info) > 0:
    desired_features = ["name", "id", "accountId", "puuid", "summonerLevel"]
    new_accounts = pd.DataFrame(new_summoner_info)
    new_accounts = new_accounts[desired_features]
    
    # combine two new and the old account information
    accounts = pd.concat([accounts, new_accounts], ignore_index=True)
    
    # save progress
    accounts.to_csv("./data/accounts/accounts_with_ids.csv", index=False)
else:
    print("No new challenger ranked players.")

No new challenger ranked players.


# Match History
----
Now that the `puuid` numbers have been pulled from the Summoner-V4 API the Match-V5 API can be used to get the match history associated with the `puuid` numbers.

`get_match_history_json` returns 100 ranked matches per puuid every 2 seconds by default.

In [113]:
#easy access for reading in account info
accounts = pd.read_csv("./data/accounts/accounts_with_ids.csv")

### Pulling from every player
----
Every player's match history needs to be pulled contrary to other API requests where I only queried new information. This is because players have played since the last time the data was pulled and it will add more matches to the dataset. The only way to check if there is any new match information is to pull the history from all of the challenger players.

In [114]:
delay = len(accounts["puuid"]) * 2
print(f"Expected Completion Time: {estimated_completion_time(delay)}")

# returns a list of 100 match histories per player in accounts based on puuid
match_histories = [get_match_history_json(player) for player in accounts["puuid"]]

completed = datetime.datetime.now().strftime("%H:%M:%S")
print(f"Completion time: {completed}")

Expected Completion Time: 17:20:48
Completion time: 17:24:08


In [115]:
# adjust from a list of matches per player to having a list of matches
matches = []
for player_history in match_histories:
    matches.extend(player_history)

In [116]:
# remove any status errors or other anomolies from the data
matches = [match for match in matches if match.startswith("NA1")]

### Duplicates - First Pass
There are few players at Challenger Rank these players are likely to be playing against each other in the same matches. In order to reduce the number of queries to the API as well as be more time efficient due to the Riot Rate Limits that match should only be queried once rather than once per player in the challenger list.

In [117]:
print(f"Before removing duplicates there were {len(matches)} matches.")
# Remove Duplicate Matches
matches = list(set(matches))
print(f"After removing duplicates there are {len(matches)} matches.")

Before removing duplicates there were 38082 matches.
After removing duplicates there are 15078 matches.


### Duplicates - Second Pass
In order to reduce the number of queries to the API as well as be more time efficient due to the Riot Rate Limits matches we already have a history of do not need to be queried.

In [118]:
match_df = pd.read_csv("./data/matches/match_df.csv")

In [119]:
print(f"Before removing duplicates there were {len(matches)} matches.")
matches = [match for match in matches if match not in match_df["match_id"].values]
print(f"After removing duplicates there are {len(matches)} matches.")

Before removing duplicates there were 15078 matches.
After removing duplicates there are 98 matches.


# Match Json's
----
Another section of the Match-V5 API will return a detailed json object based on `matchId` numbers which was collected from the Match History section of the API.

Each match is a request so only 100 matches can be pulled every two minutes. In order to get the the thousands of matches required for this analysis it will require several hours.

In [120]:
delay = len(matches) * 2
print(f"Expected Completion Time: {estimated_completion_time(delay)}")

new_match_json = [get_match_json(match) for match in matches]

completed = datetime.datetime.now().strftime("%H:%M:%S")
print(f"Completion time: {completed}")

Expected Completion Time: 17:36:01
Completion time: 17:36:56


In [121]:
# Remove failed requests from match_json. All successful pulls have metadata
num_matches = len(new_match_json)
new_match_json = [match for match in new_match_json if "metadata" in match.keys()]
print(f"{num_matches - len(new_match_json)} failed queries were cleaned from the JSON Object")

0 failed queries were cleaned from the JSON Object


### Save Match JSON
----
 - Read in the backup match JSON.
 - Output the backup match information and the new match information
 - save the new match information to it's own file.

In [80]:
# read in backup match json
with open("./data/matches/backup_match_json.json", "r") as f:
    backup_match_json = json.load(f)

In [25]:
# update backup data
if len(backup_match_json) > 0:
    with open("./data/matches/backup_match_json.json", "w") as f:
        json.dump(backup_match_json.extend(new_match_json), f)
else:
    print("Failed to pull backup json")

TypeError: object of type 'NoneType' has no len()

In [123]:
# save new match json to match_json
with open("./data/matches/match_json.json", "w") as f:
    json.dump(new_match_json, f)

In [124]:
# read in match json
with open("./data/matches/match_json.json", "r") as f:
    match_json = json.load(f)
    
if len(match_json) != len(new_match_json):
    raise NameError("Error: Match JSON does not match the Match JSON that was just pulled.")

# DataFrame Setup
----

The following code is no longer active but was used to create the Blank Template for the DataFrame.

This method was chosen because it allows everything to be done within Python dictionaries and lists before converting into a Pandas DatFrame. Adding a single line to the end of a Pandas DateFrame is computationally intensive and it will save hours of computations by completely building the Pandas Data Structure before creating a DataFrame.

```python
def setup_match_template(json_index):
    players = match_json[json_index]["info"]["participants"]
    teams = match_json[json_index]["info"]["teams"]
    match_info = {}
    
    #Get general match information
    match_info["match_id"] = []
    match_info["game_duration"] = []
    match_info["game_mode"] = []
    match_info["game_type"] = []
    match_info["game_version"] = []
    
    # This for loop seaparates the nested participants to t#p#_{feature}
    for i, player in enumerate(players):
        pid = get_player_id(i)
        for feature in PLAYER_FEATURES:
            match_info[f"{pid}_{feature}"] = []
        for statPerk in player["perks"]["statPerks"]:
            match_info[f"{pid}_statPerk_{statPerk}"] = []
        
        #This could be done in a for loop but the keystone messes this up
        match_info[f"{pid}_perk_primaryStyle"] = []
        match_info[f"{pid}_perk_keystone"] = []
        match_info[f"{pid}_perk_primary_slot_1"] = []
        match_info[f"{pid}_perk_primary_slot_2"] = []
        match_info[f"{pid}_perk_primary_slot_3"] = []
        match_info[f"{pid}_perk_subStyle"] = []
        match_info[f"{pid}_perk_sub_slot_1"] = []
        match_info[f"{pid}_perk_sub_slot_2"] = []
    
    #This loop separates the nested list of teams into t#_{feature}
    for i, team in enumerate(teams):
        for j, ban in enumerate(team["bans"]):
            match_info[f"t{i+1}_ban{j+1}"] = []
        for objective in team["objectives"]:
            match_info[f"t{i+1}_{objective}_first"] = []
            match_info[f"t{i+1}_{objective}_kills"] = []
    match_info["t1_win"] = []
    
            
    return match_info
```

In [153]:
for k, v in get_all_match_features(match_json).items():
    if(len(v) < 98):
        print(k, v)

In [154]:
new_match_df = pd.DataFrame(get_all_match_features(match_json))

In [155]:
new_match_df

,match_id,game_duration,game_mode,game_type,game_version,t1p1_assists,t1p1_baronKills,t1p1_bountyLevel,t1p1_champExperience,t1p1_champLevel,...,t2_champion_kills,t2_dragon_first,t2_dragon_kills,t2_inhibitor_first,t2_inhibitor_kills,t2_riftHerald_first,t2_riftHerald_kills,t2_tower_first,t2_tower_kills,t1_win
0,NA1_4230136963,1583,CLASSIC,MATCHED_GAME,12.4.423.2790,3,0,1,13360,15,...,29,1,4,1,1,1,2,1,6,0
1,NA1_4230155912,1416,CLASSIC,MATCHED_GAME,12.4.423.2790,4,0,1,14309,15,...,11,0,0,0,0,1,1,1,1,1
2,NA1_4230113752,1324,CLASSIC,MATCHED_GAME,12.4.423.2790,0,0,0,9126,12,...,37,1,4,1,1,0,0,0,9,0
3,NA1_4229989246,965,CLASSIC,MATCHED_GAME,12.4.423.2790,1,0,0,5627,9,...,21,1,2,0,0,0,0,1,4,0
4,NA1_4230061256,1563,CLASSIC,MATCHED_GAME,12.4.423.2790,1,0,0,11372,13,...,32,1,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,NA1_4230301793,946,CLASSIC,MATCHED_GAME,12.4.423.2790,2,0,4,8856,12,...,2,0,0,0,0,0,0,0,0,1
94,NA1_4230271752,2077,CLASSIC,MATCHED_GAME,12.4.423.2790,8,0,5,20090,18,...,32,0,1,1,4,0,0,1,11,1
95,NA1_4230098449,1412,CLASSIC,MATCHED_GAME,12.4.423.2790,5,0,0,10899,13,...,36,1,3,1,2,0,0,1,8,0
96,NA1_4230126810,1297,CLASSIC,MATCHED_GAME,12.4.423.2790,5,0,0,13354,15,...,17,1,1,0,0,0,0,0,3,1


In [156]:
match_df = pd.read_csv("./data/matches/match_df.csv")

In [157]:
row_report = (f"There are {match_df.shape[0]} rows in the original DataFrame.\n"\
              f"There are {new_match_df.shape[0]} rows in the new DataFrame.\n"\
              f"There will be {match_df.shape[0] + new_match_df.shape[0]} rows "\
               "in the combined DataFrame")
        
if match_df.shape[1] == new_match_df.shape[1]:
    print(f"Great! There are {match_df.shape[1]} columns in both DataFrames.")
    print(row_report)
else:
    raise NameError(f"***Warning! There are {match_df.shape[1]} columns in the original DataFrame "\
          f"and only {new_match_df.shape[1]} columns in the new DataFrame***")

Great! There are 1150 columns in both DataFrames.
There are 17467 rows in the original DataFrame.
There are 98 rows in the new DataFrame.
There will be 17565 rows in the combined DataFrame


## Update Match DataFrame

In [158]:
match_df = pd.concat([match_df, new_match_df])

In [159]:
match_df.shape

(17565, 1150)

In [160]:
match_df.to_csv("./data/matches/match_df.csv", index = 0)

In [161]:
match_df = pd.read_csv("./data/matches/match_df.csv")

In [162]:
match_df.head()

,match_id,game_duration,game_mode,game_type,game_version,t1p1_assists,t1p1_baronKills,t1p1_bountyLevel,t1p1_champExperience,t1p1_champLevel,...,t2_champion_kills,t2_dragon_first,t2_dragon_kills,t2_inhibitor_first,t2_inhibitor_kills,t2_riftHerald_first,t2_riftHerald_kills,t2_tower_first,t2_tower_kills,t1_win
0,NA1_4201994783,1470,CLASSIC,MATCHED_GAME,12.2.419.1399,7,0,0,8396,11,...,47,0,1,1,2,1,1,1,10,0
1,NA1_4204324706,944,CLASSIC,MATCHED_GAME,12.3.421.3734,2,0,0,7015,10,...,17,1,1,0,0,0,0,0,2,0
2,NA1_4215791943,1629,CLASSIC,MATCHED_GAME,12.3.421.5967,5,1,4,14510,15,...,27,1,3,0,0,0,0,1,2,1
3,NA1_4200912084,1329,CLASSIC,MATCHED_GAME,12.2.419.1399,4,0,0,10220,13,...,34,1,2,1,1,0,1,1,9,0
4,NA1_4218373259,1718,CLASSIC,MATCHED_GAME,12.3.421.5967,3,0,0,14922,16,...,24,1,4,1,2,0,0,0,10,0
